# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Building offices in Canberra,Australia_**
- Build a dataframe of neighborhoods in Canberra,Australia by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open offices.
***
### 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Canberra").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
cn_df = pd.DataFrame({"Neighborhood": neighborhoodList})

cn_df.head()

,Neighborhood
0,List of Canberra suburbs
1,Suburbs of Canberra
2,"Acton, Australian Capital Territory"
3,"Ainslie, Australian Capital Territory"
4,"Amaroo, Australian Capital Territory"


In [7]:
# print the number of rows of the dataframe
cn_df.shape

(124, 1)

### 3. Get the geographical coordinates

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Canberra, Australia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in cn_df["Neighborhood"].tolist() ]

In [10]:
coords

[[-35.49999999999994, 149.0000000000001],
 [-35.49999999999994, 149.0000000000001],
 [-35.28561999999994, 149.11827000000005],
 [-35.26221999999996, 149.14655000000005],
 [-35.16921999999994, 149.12637000000007],
 [-35.25803999999994, 149.08293000000003],
 [-35.47003999999998, 149.09771000000012],
 [-35.30828999999994, 149.13354000000004],
 [-35.343219999999974, 149.21217000000001],
 [-35.23944999999998, 149.0634500000001],
 [-35.15749999999997, 149.1430600000001],
 [-35.43090999999998, 149.0802000000001],
 [-35.27461999999997, 149.1383300000001],
 [-35.24571999999995, 149.0926800000001],
 [-35.43791999999996, 149.1092000000001],
 [-35.29472999999996, 149.16005000000007],
 [-35.306539999999984, 149.12655000000007],
 [-35.16863999999998, 149.09315000000004],
 [-35.35617999999994, 149.0407600000001],
 [-35.19797999999997, 149.03235000000006],
 [-35.352499999999964, 149.0754700000001],
 [-35.41901999999993, 149.1237400000001],
 [-35.28006999999997, 149.13093000000003],
 [-35.2794899999999

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
cn_df['Latitude'] = df_coords['Latitude']
cn_df['Longitude'] = df_coords['Longitude']

In [13]:
# check the neighborhoods and the coordinates
print(cn_df.shape)
cn_df

(124, 3)


,Neighborhood,Latitude,Longitude
0,List of Canberra suburbs,-35.500000,149.000000
1,Suburbs of Canberra,-35.500000,149.000000
2,"Acton, Australian Capital Territory",-35.285620,149.118270
3,"Ainslie, Australian Capital Territory",-35.262220,149.146550
4,"Amaroo, Australian Capital Territory",-35.169220,149.126370
5,"Aranda, Australian Capital Territory",-35.258040,149.082930
6,"Banks, Australian Capital Territory",-35.470040,149.097710
7,"Barton, Australian Capital Territory",-35.308290,149.133540
8,"Beard, Australian Capital Territory",-35.343220,149.212170
9,"Belconnen, Australian Capital Territory",-35.239450,149.063450


In [14]:
# save the DataFrame as CSV file
cn_df.to_csv("cn_df.csv", index=False)

### 4. Create a map of Canberra with neighborhoods superimposed on top

In [15]:
# get the coordinates of Canberra
address = 'Canberra, Australia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Canberra, Australia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Canberra, Australia -35.2975906, 149.1012676.


In [16]:
# create map of Canberra using latitude and longitude values
map_cn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(cn_df['Latitude'], cn_df['Longitude'], cn_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_cn)  
    
map_cn

In [17]:
# save the map as HTML file
map_cn.save('map_cn.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [18]:
# define Foursquare Credentials and Version
CLIENT_ID = 'L4DUAZGNQWLFPRN4GS1XSVFEZZPP0ZWW23Y5TF2VKLQHKV1T' # your Foursquare ID
CLIENT_SECRET = 'Q33WDJORAYW22BUBJBF5CCXFOVEU1LDQGW2RY5FEIIXQLYSX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L4DUAZGNQWLFPRN4GS1XSVFEZZPP0ZWW23Y5TF2VKLQHKV1T
CLIENT_SECRET:Q33WDJORAYW22BUBJBF5CCXFOVEU1LDQGW2RY5FEIIXQLYSX


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(cn_df['Latitude'], cn_df['Longitude'], cn_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3626, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of Canberra suburbs,-35.50000,149.00000,Now Flex,-35.499531,149.000090,Clothing Store
1,List of Canberra suburbs,-35.50000,149.00000,Office Furniture Network,-35.490254,148.987581,Construction & Landscaping
2,Suburbs of Canberra,-35.50000,149.00000,Now Flex,-35.499531,149.000090,Clothing Store
3,Suburbs of Canberra,-35.50000,149.00000,Office Furniture Network,-35.490254,148.987581,Construction & Landscaping
4,"Acton, Australian Capital Territory",-35.28562,149.11827,Palace Electric Cinema,-35.285014,149.123135,Movie Theater


**Let's check how many venues were returned for each neighorhood**

In [21]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Acton, Australian Capital Territory",100,100,100,100,100,100
"Ainslie, Australian Capital Territory",69,69,69,69,69,69
"Amaroo, Australian Capital Territory",27,27,27,27,27,27
"Aranda, Australian Capital Territory",23,23,23,23,23,23
"Banks, Australian Capital Territory",6,6,6,6,6,6
"Barton, Australian Capital Territory",93,93,93,93,93,93
"Beard, Australian Capital Territory",5,5,5,5,5,5
"Belconnen, Australian Capital Territory",75,75,75,75,75,75
"Bonner, Australian Capital Territory",5,5,5,5,5,5


**Let's find out how many unique categories can be curated from all the returned venues**

In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 197 uniques categories.


In [23]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Clothing Store', 'Construction & Landscaping', 'Movie Theater',
       'Concert Hall', 'Hotel Bar', 'Coffee Shop', 'Café',
       'History Museum', 'Comic Shop', 'Wine Bar', 'Botanical Garden',
       'Park', 'Italian Restaurant', 'Trail', 'Tapas Restaurant',
       'Lounge', 'Burrito Place', 'Whisky Bar', 'Thai Restaurant',
       'Restaurant', 'Gaming Cafe', 'Museum', 'Beer Bar', 'Record Shop',
       'Bakery', 'Brewery', 'Cocktail Bar', 'Chocolate Shop', 'Library',
       'Food Truck', 'Japanese Restaurant', 'Science Museum',
       'Asian Restaurant', 'Steakhouse', 'Burger Joint',
       'Electronics Store', 'Molecular Gastronomy Restaurant', 'Theater',
       'Tea Room', 'Hotel', 'Tiki Bar', 'French Restaurant',
       'Mediterranean Restaurant', 'Bar', 'Indian Restaurant',
       'Vietnamese Restaurant', 'Shopping Mall', 'Gym', 'Noodle House',
       'Szechuan Restaurant'], dtype=object)

In [24]:
# check if the results contain "Store"
#"Store" in venues_df['VenueCategory'].unique()

### 6. Analyze Each Neighborhood

In [25]:
# one hot encoding
cn_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cn_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cn_onehot.columns[-1]] + list(cn_onehot.columns[:-1])
cn_onehot = cn_onehot[fixed_columns]

print(cn_onehot.shape)
cn_onehot.head()

(3626, 198)


,Neighborhoods,Airport,Airport Lounge,Airport Terminal,Animal Shelter,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Botanical Garden,Bowling Alley,Brewery,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Campaign Office,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Library,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Newsstand,Noodle House,North Indian Restaurant,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Public Art,RV Park,Racecourse,Record Shop,Rental Car Location,Restaurant,River,Rugby Pitch,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Soccer Field,Social Club,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Winery,Yoga Studio,Zoo
0,List of Canberra suburbs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,List of Canberra suburbs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Suburbs of Canberra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [26]:
cn_grouped = cn_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(cn_grouped.shape)
cn_grouped

(124, 198)


,Neighborhoods,Airport,Airport Lounge,Airport Terminal,Animal Shelter,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Botanical Garden,Bowling Alley,Brewery,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Campaign Office,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Library,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Newsstand,Noodle House,North Indian Restaurant,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Public Art,RV Park,Racecourse,Record Shop,Rental Car Location,Restaurant,River,Rugby Pitch,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Soccer Field,Social Club,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Winery,Yoga Studio,Zoo
0,"Acton, Australian Capital Territory",0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.0,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.010000,0.01,0.000000,0.010000,0.000,0.000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.130000,0.000000,0.010000,0.01,0.000000,0.010000,0.100000,0.01,0.01,0.0,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.050000,0.010000,0.000000,0.010000,0.000000,0.030000,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.0000,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.010000,0.000,0.000000,0.010

In [29]:
len(cn_grouped[cn_grouped["Building"] > 0])

3

**Create a new DataFrame for commercial office data only**

In [30]:
cn_mall = cn_grouped[["Neighborhoods","Building"]]

In [33]:
cn_mall.head()

,Neighborhoods,Building
0,"Acton, Australian Capital Territory",0.0
1,"Ainslie, Australian Capital Territory",0.0
2,"Amaroo, Australian Capital Territory",0.0
3,"Aranda, Australian Capital Territory",0.0
4,"Banks, Australian Capital Territory",0.0


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Canberra into 3 clusters.

In [34]:
# set number of clusters
kclusters = 3

cn_clustering = cn_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cn_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0])

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
cn_merged = cn_mall.copy()

# add clustering labels
cn_merged["Cluster Labels"] = kmeans.labels_

In [36]:
cn_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
cn_merged.head()

,Neighborhood,Building,Cluster Labels
0,"Acton, Australian Capital Territory",0.0,0
1,"Ainslie, Australian Capital Territory",0.0,0
2,"Amaroo, Australian Capital Territory",0.0,0
3,"Aranda, Australian Capital Territory",0.0,0
4,"Banks, Australian Capital Territory",0.0,0


In [37]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cn_merged = cn_merged.join(cn_df.set_index("Neighborhood"), on="Neighborhood")

print(cn_merged.shape)
cn_merged.head() # check the last columns!

(124, 5)


,Neighborhood,Building,Cluster Labels,Latitude,Longitude
0,"Acton, Australian Capital Territory",0.0,0,-35.28562,149.11827
1,"Ainslie, Australian Capital Territory",0.0,0,-35.26222,149.14655
2,"Amaroo, Australian Capital Territory",0.0,0,-35.16922,149.12637
3,"Aranda, Australian Capital Territory",0.0,0,-35.25804,149.08293
4,"Banks, Australian Capital Territory",0.0,0,-35.47004,149.09771


In [48]:
# sort the results by Cluster Labels
print(cn_merged.shape)
cn_merged.sort_values(["Cluster Labels"], inplace=True)
cn_merged
cn_merged.to_csv("cn_merged.csv", index=False)

(124, 5)


**Finally, let's visualize the resulting clusters**

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cn_merged['Latitude'], cn_merged['Longitude'], cn_merged['Neighborhood'], cn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [41]:
cn_merged.loc[cn_merged['Cluster Labels'] == 0]

,Neighborhood,Building,Cluster Labels,Latitude,Longitude
0,"Acton, Australian Capital Territory",0.0,0,-35.285620,149.118270
89,"Nicholls, Australian Capital Territory",0.0,0,-35.184180,149.099160
88,"Ngunnawal, Australian Capital Territory",0.0,0,-35.173190,149.108020
87,"Narrabundah, Australian Capital Territory",0.0,0,-35.331220,149.152400
86,"Moncrieff, Australian Capital Territory",0.0,0,-35.162000,149.113490
85,"Monash, Australian Capital Territory",0.0,0,-35.415160,149.092260
84,"Molonglo, Australian Capital Territory",0.0,0,-35.329573,149.175896
83,"Mitchell, Australian Capital Territory",0.0,0,-35.219320,149.134420
82,"Melba, Australian Capital Territory",0.0,0,-35.213620,149.053120
81,"McKellar, Australian Capital Territory",0.0,0,-35.216570,149.075700


#### Cluster 1

In [42]:
cn_merged.loc[cn_merged['Cluster Labels'] == 1]

,Neighborhood,Building,Cluster Labels,Latitude,Longitude
123,"Yarralumla, Australian Capital Territory",0.033333,1,-35.30589,149.1067


#### Cluster 2

In [45]:
cn_merged.loc[cn_merged['Cluster Labels'] == 2]

,Neighborhood,Building,Cluster Labels,Latitude,Longitude
5,"Barton, Australian Capital Territory",0.010753,2,-35.30829,149.13354
95,"Parkes, Australian Capital Territory",0.010000,2,-35.30258,149.12881


#### Observations:
Most of the shopping malls are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.